# Logistic regression - comparing samplers

### Import libraries

In [9]:
import tensorflow as tf
import numpy as np
from samplers import *
from utils import *
import matplotlib.pyplot as plt
%matplotlib inline 
#np.random.seed(0)
#tf.set_random_seed(0)
from sklearn import linear_model
import copy
import scipy.stats
import itertools
import os
import time



### Dar data generation

In [8]:
# Generate logistic regression data
D =   20
N = 1000
target_w = 10*(np.random.rand(D) - .5)
train_images = np.random.rand(N,D) - .5
cov = np.diag(np.ones(D))
cov[0,0] = 6
for i in range(N):
    train_images[i,:] = np.random.multivariate_normal(np.zeros(D),cov)
base=np.float32
tfbase=tf.float32
train_probs = logit(np.dot(train_images,target_w))
train_labels = np.expand_dims(train_probs<np.random.rand(N),1).astype(base)
xData = np.float32( train_images.reshape( train_labels.shape[0], train_images[0].size ) )
yData = train_labels
probs=logit(np.matmul(train_images, target_w))
print((NLLexplicit(probs,train_probs)))

NLLexplicit(probs,train_probs)


nan


nan

In [3]:
# Use SKLearn to run SAG
import sklearn.linear_model as lin_mod
maxNumEpochs=5000;
skLogistic=lin_mod.LogisticRegression(fit_intercept=False,solver='sag',C=1e10,max_iter=N*10,tol=1e-6)
skLogistic.fit(xData,np.reshape(yData,[N,]))
W_map=skLogistic.coef_.copy()
probs_sk=logit(np.matmul(train_images, np.reshape(W_map,[D,1])))
print((NLLexplicit(probs_sk,np.reshape(yData,[N]))))
print(W_map-target_w) # show differences in W
W_map = np.transpose(W_map)

[-inf]
[[ 0.32002982  1.39770253 -0.97311093  1.39519925 -0.21466235 -0.99232534
   1.42017851  0.9662785   0.84334037  1.00422016  0.90446579  0.41954094
   0.89402187  0.40207118 -0.40824313 -0.51620558 -0.54209306  0.67453355
  -0.00803189  0.34644805]]


### Model Statement

In [4]:
tf.reset_default_graph()
xDataPH = tf.placeholder(tfbase, [None,D])
yDataPH = tf.placeholder(tfbase, [None,1],)
#yClass = tf.placeholder(tf.float32, [None, 10])
W = tf.Variable(tf.zeros([D,1],tfbase),tfbase)
linProd= tf.matmul(xDataPH,W)
objective=tf.nn.sigmoid_cross_entropy_with_logits(linProd,yDataPH)
NLL=tf.reduce_mean(objective)
# Create an ExponentialMovingAverage object

### Define operations for custom learning rule

In [5]:
# operation to calculate gradient and return gradient
opt = tf.train.GradientDescentOptimizer(learning_rate=1)
grads_and_vars = opt.compute_gradients(NLL)
gradients, variables = zip(*grads_and_vars)
g=tf.reshape(gradients[0],[D])
weights = tf.reshape(variables[0],[D])
ghostGradient = tf.placeholder(tfbase,[D])
grads_to_apply = []
grads_to_apply.append(tf.reshape(ghostGradient,[D,1]))
grad_and_vars_to_apply=list(zip(grads_to_apply, variables))
train_step = opt.apply_gradients(grad_and_vars_to_apply)

### Explicit gradients so that variance can be calculate for the GP
#### Note this is *not* general purpose and will not work if the model statement is changed

In [6]:
velocity_dummy=tf.placeholder(tfbase,[D])
yPred=tf.sigmoid(linProd)
xPHR=tf.reshape(xDataPH,[-1,D])
gradient_explicit=-(yDataPH-yPred)*xPHR
gradient_calculation=tf.reduce_mean(gradient_explicit,0)
gradient_variance_calculation=tf.reduce_mean(gradient_explicit*gradient_explicit,0) \
    -gradient_calculation*gradient_calculation
better_gradient_variance_intermediate=tf.matmul(gradient_explicit,tf.reshape(velocity_dummy,[D,1]))
better_gradient_variance_calculation=tf.nn.moments(better_gradient_variance_intermediate,0)[1]
hessian_explicit= tf.reshape(yPred*(1-yPred),[-1,1,1])*(tf.reshape(xPHR,[-1,D,1])*tf.reshape(xPHR,[-1,1,D]))
hessian_calculation=tf.reduce_mean(hessian_explicit,0)

### Define a method to calculate accuracy from a minibatch

In [7]:
correct_prediction = tf.equal(yDataPH, tf.cast(.5+.5*tf.sign(tf.matmul(xDataPH, W)),tfbase))
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tfbase))

In [8]:
hess = np.zeros((D,D))
for i in range(N):
    ex = np.exp(np.dot(np.squeeze(W_map),train_images[i,:]))
    hess += (ex/float((1+ex)**2))*np.outer(train_images[i,:],train_images[i,:])

### function to initialize session and run optimization/sampling

In [9]:
def runOptimizationExplicit(my_updater,use_preconditioner,W_map=0):
    start = time.time()
    # initialize tf for optimization
    init = tf.initialize_all_variables()
    sess = tf.Session()
    sess.run(init)
    lam=1e-4
    beta1=.99
    grad2=np.zeros(D)
    preconditioner = 0
    perrs = []
    # how long to run optimization
    N=len(xData)
    # iteration counter
    iter=0
    # initialize variables to store history
    NLL_factor = 5
    n_i = 0
    NLL_store = np.zeros(total_iter/NLL_factor)
    acc_store=np.zeros(total_iter)
    samples = np.zeros((total_iter,D))
    if not type(W_map) == int:
        wset=W.assign(W_map)
        with sess.as_default():
            sess.run(wset)
        #print 'Initial position assigned, 0 coordinate is  - ', W_map[0]
        
    if not use_preconditioner:
        preconditioner = 1
    # main optimization loop
    for n in xrange(n_epochs):
        for minibatch in iterate_minibatches(xData, yData, batch_size, shuffle=True):
            
            # grab new minibatch
            x_batch, y_batch = minibatch
            
            # calculate gradient
            if my_updater.__class__ == lSBPS:
                velocity_input=my_updater.v
                with sess.as_default():
                    gradient=gradient_calculation.eval(feed_dict={xDataPH: x_batch, yDataPH: y_batch})
                    
                    # build preconditioner 
                    if use_preconditioner:
                        grad2=gradient*gradient*(1-beta1)+beta1*grad2
                        preconditioner=1. /(np.sqrt(grad2)+lam)
                        preconditioner = preconditioner / float(np.mean(preconditioner))
                        
                    gradient_times_velocity_variance=better_gradient_variance_calculation.eval( \
                        feed_dict={xDataPH: x_batch, yDataPH: y_batch, velocity_dummy: preconditioner*velocity_input})
                                                                                               
                # apply some operation to the gradient
                new_gradient=my_updater.update(preconditioner*gradient,gradient_times_velocity_variance)
            else:
                with sess.as_default():
                    gradient=g.eval(feed_dict={xDataPH: x_batch, yDataPH: y_batch})
                new_gradient=my_updater.update(gradient)  
            # apply gradient and track quantities
            with sess.as_default():
                # apply the new quantity to the parameters
                train_step.run(feed_dict={ghostGradient: preconditioner*new_gradient})
                
                if my_updater.__class__ == lSBPS:
                    my_updater.all_vs[-1] = preconditioner*my_updater.all_vs[-1]
                    perrs.append(my_updater.p_errs)
                # store samples
                samples[iter,:] = weights.eval(feed_dict={xDataPH: x_batch, yDataPH: y_batch})
        
            if np.mod(iter,NLL_factor) == 0:
                NLL_store[n_i]=sess.run(NLL, feed_dict={xDataPH: xData, yDataPH: yData})
                #print 'Evaluating NLL - ', NLL_store[n_i]
                n_i += 1
            # update iteration counter
            iter+=1
    end = time.time()
    print 'Runtime :', end - start
    return NLL_store,acc_store,samples

# SBPS

In [10]:
def generate_SBPS_samples(c_SBPS,W_map,W,N):
    # initialize tf for optimization
    print 'Generating discrete samples and calculating NLL'
    init = tf.initialize_all_variables()
    sess = tf.Session()
    sess.run(init)
    # how long to run optimization
    time_step = c_SBPS.total_time/total_iter
    j = 0
    n_i = 0
    NLL_store = np.zeros(total_iter/NLL_factor)
    samples = np.zeros((total_iter,c_SBPS.D))
    total_time_moved = 0
    all_curr_ts = []
    i = 0
    time_until_next_sample = time_step
    wset=W.assign(W_map)
    with sess.as_default():
        sess.run(wset)

    while i < total_iter - 1:
        curr_time = c_SBPS.all_times[j]
        while curr_time > time_until_next_sample:
            curr_time -= time_until_next_sample
            g = time_until_next_sample*c_SBPS.all_vs[j]
            with sess.as_default():
                train_step.run(feed_dict={ghostGradient: g})
                if i < total_iter:
                    samples[i,:] = weights.eval(feed_dict={xDataPH: xData, yDataPH: yData})
                if np.mod(i,NLL_factor) == 0 and i < total_iter:
                    NLL_store[n_i]=sess.run(NLL, feed_dict={xDataPH: xData, yDataPH: yData})
                    #print 'Evaluating NLL - ', NLL_store[n_i]
                    n_i += 1
            i += 1
            time_until_next_sample = time_step

        # moving remainder
        time_until_next_sample -= curr_time
        g = curr_time*c_SBPS.all_vs[j]
        with sess.as_default():
            train_step.run(feed_dict={ghostGradient: g})
        j += 1
    return NLL_store,samples

In [11]:
# Calculate Hessian
init = tf.initialize_all_variables()
sess = tf.Session()
sess.run(init)
#if D < 10:
Wdummy=tf.placeholder(tfbase,[D,1])
Wset=W.assign(Wdummy)
with sess.as_default():
    sess.run(Wset,feed_dict={Wdummy: W_map})
    hessian_map=hessian_calculation.eval(feed_dict={xDataPH: xData, yDataPH: yData})
hessian_map=hessian_map*N
cov_map=np.linalg.inv(hessian_map)
print np.linalg.eig(hessian_map)[0]

[ 151.89334106    0.29898757   40.99404907   36.25157547   35.4541626
   31.39330292   30.95247841   11.10289001   28.33082962   27.00761223
   12.35878754   13.82955933   15.6628437    16.03650856   17.0235157
   24.48535347   22.73886871   21.38258171   20.34235001   19.67732048]


## Run samplers

In [15]:
# run samplers
def run_sam():
    n_epochs=100
    batch_size=100
    init = tf.initialize_all_variables()
    sess = tf.Session()
    sess.run(init)
    total_iter=n_epochs*(N/batch_size)
    print('Running ' + str(n_epochs) + ' epochs (' +str(total_iter) + ' iterations) per method')
    fW_map = np.ndarray.flatten(W_map)

    W_start = np.random.multivariate_normal(fW_map,cov_map*10)
    W_start = np.expand_dims(W_start,1)
    k = 3
    step_size = .1
    labels = []
    trajectories = []
    NLLs = []
    mbs = []

    # lSBPS
    k = 3
    print('Running lSBPS, k = ' + str(k))
    my_lSBPS = lSBPS(D,N,batch_size, [0,0],k)
    my_lSBPS.reset_counter()
    NLL_lSBPS_0,acc_lSBPS,samples_lSBPS_0 = runOptimizationExplicit(my_lSBPS,0,W_start)
    NLL_lSBPS,samples_lSBPS = generate_SBPS_samples(my_lSBPS,W_start,W,N)
    print('Number of bounces: ' + str(my_lSBPS.num_bounce))
    num_pacc = (my_lSBPS.p_errs/float(my_lSBPS.num_bounce))
    print('percent p(acc)>1: ', num_pacc)
    reject = (len(samples_lSBPS[:,0]) - my_lSBPS.num_bounce)/float(len(samples_lSBPS[:,0]))
    print('percent rejections: ', reject)
    print('percent negative slope: ', my_lSBPS.neg_S/float(len(my_lSBPS.all_Gs)))
    print 'total travel time: ', my_lSBPS.total_time 
    labels.append('SBPS')
    trajectories.append(samples_lSBPS)
    NLLs.append(NLL_lSBPS)
    mbs.append(batch_size)

    ## Run SGLD
    stepsizes = [1,.1,.01]
    for i in range(len(stepsizes)):
        print('Running SGLD ', stepsizes )
        total_iter=n_epochs*(N/batch_size)
        my_SGLD = SGLD(N,stepsizes[i])
        NLL_SGLD,acc_SGLD,samples_SGLD = runOptimizationExplicit(my_SGLD,0,W_start)
        labels.append('SGLD ' + str(stepsizes[i]))
        trajectories.append(samples_SGLD)
        NLLs.append(NLL_SGLD)
        mbs.append(batch_size)
    '''
    # Run mSGNHT
    batch_size=100
    total_iter=n_epochs*(N/batch_size)
    my_mSGNHT = mSGNHT(D,N,.1)
    print 'Running mSGNHT'
    NLL_mSGNHT,acc_mSGNHT,samples_mSGNHT = runOptimizationExplicit(my_mSGNHT,0,W_start)
    labels.append('mSGNHT')
    trajectories.append(samples_mSGNHT)
    NLLs.append(NLL_mSGNHT)
    mbs.append(batch_size)

    # lipschitz BPS
    print 'Running lipSBPS'
    batch_size=1
    total_iter=n_epochs*(N/batch_size)
    verbose=False
    L=np.sqrt(D)*np.max(np.abs(xData)) # this could be tightened, probably by a factor of 2
    my_bouncer = lipschitzSBPS(D,N,L,verbose)
    NLL_lipSBPS0,acc_lipSBPS,samples_lipSBPS0 = runOptimizationExplicit(my_bouncer,0,W_start)
    NLL_lipSBPS0 = 0
    acc_lipSBPS = 0
    my_bouncer.all_vs = [-v for v in my_bouncer.all_vs]
    NLL_lipSBPS,samples_lipSBPS = generate_SBPS_samples(my_bouncer,W_start,W,N)
    labels.append('lipSBPS')
    trajectories.append(samples_lipSBPS)
    NLLs.append(NLL_lipSBPS)
    mbs.append(batch_size)

    # Run ZZ-SS
    batch_size=1
    total_iter=n_epochs*(N/batch_size)
    logisticRegressionM=np.max(xData)
    my_zigzag_ss=ZZ_SS(D,N,logisticRegressionM)
    my_zigzag_ss.reset_counter()
    print('Running ZZ-SS')
    NLL_zz_ss0,acc_zz_ss,samples_zz0 = runOptimizationExplicit(my_zigzag_ss,0,W_start)
    acc_zz_ss = 0
    NLL_zz_ss0 = 0
    print('Number of flips: ' + str(my_zigzag_ss.flipcounter))
    NLL_zz_ss,samples_zz = generate_SBPS_samples(my_zigzag_ss,W_start,W,N)
    labels.append('ZZ-SS')
    trajectories.append(samples_zz)
    NLLs.append(NLL_zz_ss)
    mbs.append(1)
    '''
    wset=W.assign(W_map)
    with sess.as_default():
        sess.run(wset)
    print 'MAP assigned, x coordinate is  - ', W_start[0]    
    NLL_map = sess.run(NLL, feed_dict={xDataPH: xData, yDataPH: yData})
    cond_num = np.max(np.linalg.eig(hessian_map)[0])/float(np.min(np.linalg.eig(hessian_map)[0]))
    print 'condition number ', cond_num
    stats = [total_iter,cond_num,my_lSBPS.num_bounce,my_lSBPS.p_errs/float(my_lSBPS.num_bounce),\
            (len(samples_lSBPS[:,0]) - my_lSBPS.num_bounce)/float(len(samples_lSBPS[:,0])),\
            my_lSBPS.neg_S/float(len(my_lSBPS.all_Gs)),my_lSBPS.total_time]
            # my_plSBPS.num_bounce,my_plSBPS.p_errs/float(my_plSBPS.num_bounce),\
            #(len(samples_plSBPS[:,0]) - my_plSBPS.num_bounce)/float(len(samples_plSBPS[:,0])),\
            #my_plSBPS.neg_S/float(len(my_plSBPS.all_Gs)),my_plSBPS.total_time]
    #np.save('samples_' + str(D),[stats,trajectories,NLLs,labels,NLL_map])
    #print 'saved samples'

In [19]:
import profile
n_epochs=100
batch_size=100
total_iter=n_epochs*(N/batch_size)
NLL_factor = 5
profile.run('run_sam()')

Running 100 epochs (1000 iterations) per method
Running lSBPS, k = 3
Runtime : 34.5081648827
Generating discrete samples and calculating NLL
Number of bounces: 146
('percent p(acc)>1: ', 0.06164383561643835)
('percent rejections: ', 0.854)
('percent negative slope: ', 0.393)
total travel time:  19.0200820379
('Running SGLD ', [1, 0.1, 0.01])
Runtime : 20.9941210747
('Running SGLD ', [1, 0.1, 0.01])
Runtime : 22.2493648529
('Running SGLD ', [1, 0.1, 0.01])
Runtime : 21.8827528954
MAP assigned, x coordinate is  -  [ 2.05440832]
condition number  508.025609134
         4260479 function calls (4220891 primitive calls) in 58.908 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        6    0.000    0.000    0.000    0.000 :0(TF_CloseSession)
    17009    0.119    0.000    0.119    0.000 :0(TF_DeleteBuffer)
        6    0.012    0.002    0.012    0.002 :0(TF_DeleteSession)
        6    0.000    0.000    0.000    0.000 :0(TF_Delete

## Plots

In [ ]:
# make plots pretty
plt.rcParams['font.size'] = 16
plt.rcParams['font.family']='serif'
plt.rcParams['mathtext.default']='regular'
plt.figure(figsize=(12,12))

# Trajectories 
def gauss(x,y,hess,mu):
    vec = np.asarray([x,y]) - np.asarray(mu)
    return  -np.dot(vec,np.dot(hess,np.transpose(vec)))/2.0
#x = np.linspace(0.4, .6, 100) #N=60000
#y = np.linspace(2, 2.4, 100)
x = np.linspace(W_map[0]-6*np.sqrt(cov_map[0,0]), W_map[0]+6*np.sqrt(cov_map[0,0]), 100)
y = np.linspace(W_map[1]-6*np.sqrt(cov_map[1,1]), W_map[1]+6*np.sqrt(cov_map[1,1]), 100)
X, Y = np.meshgrid(x, y)
zs = np.array([gauss(x,y,np.linalg.inv(cov_map[0:2,0:2]),[W_map[0][0],W_map[1][0]]) \
               for x,y in zip(np.ravel(X), np.ravel(Y))])
Z = zs.reshape(X.shape)
CL = plt.contour(X, Y, Z,np.arange(-30, -5, 5))
plt.clabel(CL, inline=1, fontsize=10)

plt.title('Trajectories')
ptx = []
pty = []
for i in range(10):
    tx = np.random.multivariate_normal(np.ndarray.flatten(W_map),cov_map)
    ptx.append(tx[0])
    pty.append(tx[1])
    #ptx.append(np.dot(tx,base_change)[0])
    #pty.append(np.dot(tx,base_change)[1])

plt.legend(loc='upper left')

for i in range(len(trajectories)):
    plt.plot(trajectories[i][:-1,0],trajectories[i][:-1,1],label=labels[i])
#
plt.xlabel('coordinate 0')
plt.ylabel('coordinate 1')
plt.scatter(ptx,pty,color='purple',label='Laplace samples')
plt.scatter(W_map[0],W_map[1],label='MAP',color='red')
plt.legend(loc=4)

In [ ]:
def ACF(data,burnin):
    corrs = []
    for i in range(len(data)): 
        # empirical mean
        #mean = []
        #for k in range(total_iter-burnin):
        #    mean.append(np.mean(trajectories[i][burnin:burnin+k,0]))
        #means.append(mean)

        #ACF
        sig = data[i][burnin:,0] - np.mean(data[i][burnin:,0]) 
        corr = np.correlate(sig,sig,'full') 
        corr = corr/np.max(corr)
        corr = corr[corr.shape[0]/2:] 
        corrs.append(corr)
    return corrs

In [ ]:
def rotate_traj_ACF(trajectories,burnin):
    base_change = np.linalg.eig(cov_map)[1]
    corr2s = []
    inds = [np.linalg.eig(cov_map)[0].argmax(),np.linalg.eig(cov_map)[0].argmin()]
    wtrajectories = []
    for i in range(len(trajectories)):
        wtrajectories.append(np.dot(trajectories[i],base_change))
    wwmap = np.squeeze(np.dot(np.transpose(W_map),base_change))
    for j in range(2):
        corrs3 = [] 
        for i in range(len(trajectories)): 
            #ACF
            sig = wtrajectories[i][burnin/mbs[i]:,inds[j]] - np.mean(wtrajectories[i][burnin/mbs[i]:,inds[j]]) 
            corr = np.correlate(sig,sig,'full') 
            corr = corr/np.max(corr)
            corr = corr[corr.shape[0]/2:] 
            corrs3.append(corr)
        corr2s.append(corrs3)
    return wtrajectories, corr2s, wwmap
#[wtrajectories, corr2s, wwmap] = rotate_traj_ACF(trajectories,burnin)

In [ ]:
def all_plots(NLLs,labels,fname,corr2s,wtrajectories,wwmap,mbs):
    plt.rcParams['font.size'] = 16
    plt.rcParams['font.family']='serif'
    plt.rcParams['mathtext.default']='regular'
    inds = [np.linalg.eig(cov_map)[0].argmax(),np.linalg.eig(cov_map)[0].argmin()]
    #lines = ["-","--","-.",":"]
    lines = ["-"]
    alphas = [1,1,1,1,1,1,1]
    linecycler = itertools.cycle(lines)
    colors = ['b','g','r','y','m','orange','c']

    # NLL plot
    from matplotlib.ticker import FormatStrFormatter,AutoMinorLocator

    chi_sq_m = D/float(2*N)
    chi_sq_s = np.sqrt(D/float(2*N**2))
    chi_sq_mode = max((D-2)/float(2*N),0)
    plt.subplots(figsize=(20,10))
    plt.subplot(231)

    plt.title('NLL per data point (log scale)')
    for j in range(len(NLLs)):
        plt.loglog([mbs[j]*NLL_factor*i for i in range(len(NLLs[j]))],NLLs[j],\
                   next(linecycler),label=labels[j],color = colors[j],alpha = alphas[j])
    max_len = mbs[0]*NLL_factor*len(NLLs[0])
    min_len = NLL_factor*max(mbs)
    plt.loglog([min_len,max_len],[NLL_map,NLL_map],label='NLL $_{\hat{w}}/N$',color='black',linestyle=':')
    plt.loglog([min_len,max_len],[NLL_map+chi_sq_m+chi_sq_s,NLL_map+chi_sq_m+chi_sq_s],color='black',linestyle='--',lw=2)
    plt.loglog([min_len,max_len],[NLL_map+chi_sq_m-chi_sq_s,NLL_map+chi_sq_m-chi_sq_s],color='black',linestyle='--',lw=2)
    plt.loglog([min_len,max_len],[NLL_map+chi_sq_m,NLL_map+chi_sq_m],\
                 label='Laplace approx.',color='black',lw=2)
    #             label=r'$\frac{1}{N}(\frac{1}{2}\chi^2(d)+NLL_{\hat{w}})$',color='orange',lw=2)
    plt.xlabel('data points observed')

    ax = plt.gca()
    #ax.set_yscale('log')
    #plt.tick_params(axis='y', which='minor')
    # ax.yaxis.set_minor_formatter(FormatStrFormatter("%.2f"))
    #minor_locator = AutoMinorLocator(3)
    #ax.yaxis.set_minor_locator(minor_locator)
    plt.legend(bbox_to_anchor=(0, -.25), loc=2)
    print max_len, NLL_map
    plt.axis([min_len,max_len,0.95*NLL_map,1.05*np.max([np.max(NLL) for NLL in NLLs])])

    # ACF
    total_iter = max([len(wtrajectories[i])*mbs[i] for i in range(len(wtrajectories))])
    j = 1
    plt.subplot(232)
    plt.title('direction of largest covariance')
    for i in range(len(wtrajectories)): 
        x = np.asarray(range(len(corr2s[0][i])))
        plt.semilogx(mbs[i]*x,corr2s[0][i],next(linecycler),label=labels[i],color = colors[i],alpha = alphas[i]) 
    plt.xlabel('lag')    
    plt.ylabel('ACF')
    plt.grid()
    plt.axis([max(mbs)*10,total_iter-burnin,np.min([min(c) for c in corr2s[0]]),1])

    # ACF
    j = 1
    plt.subplot(233)
    plt.title('direction of smallest covariance')
    for i in range(len(wtrajectories)): 
        x = np.asarray(range(len(corr2s[1][i])))
        plt.semilogx(mbs[i]*x,corr2s[1][i],label=labels[i],color = colors[i],alpha = alphas[i]) 
    plt.xlabel('lag')    
    plt.grid()
    plt.axis([max(mbs)*10,total_iter-burnin,np.min([min(c) for c in corr2s[1]]),1])

    # Coordinates (largest)
    def plot_coord(j):
        rng = 5*total_iter/5
        std = np.sqrt(np.linalg.eig(cov_map)[0])[inds[j]]
        for i in range(len(wtrajectories)): 
            x = np.asarray(range(len(wtrajectories[i][:rng/mbs[i],:])))
            plt.plot(mbs[i]*x,wtrajectories[i][:rng/mbs[i],inds[j]],next(linecycler),\
                     color = colors[i],alpha = alphas[i])
        plt.plot([0,rng],[wwmap[inds[j]],wwmap[inds[j]]],color='black',lw=2)
        plt.plot([0,rng],[wwmap[inds[j]] + std,wwmap[inds[j]] + std],color='black',linestyle='--',lw=2)
        plt.plot([0,rng],[wwmap[inds[j]] - std,wwmap[inds[j]] - std],color='black',linestyle='--',lw=2)
        plt.xlabel('data points observed') 
        plt.locator_params(axis='x',nbins=4)
        plt.ticklabel_format(style='sci', axis='x', scilimits=(0,0))

    plt.subplot(235)
    plot_coord(0)
    plt.ylabel('coordinate trace')
    #plt.ylim(15,35)
    plt.subplot(236)
    plot_coord(1)
    std = np.sqrt(np.linalg.eig(cov_map)[0])[inds[j]]
    plt.axis('tight')
    plt.xlim(0,total_iter/20)

    plt.tight_layout()
    savefig(fname + '_d='+str(D),'../paper/arxiv/figures')

In [ ]:
def savefig(name,dest):
    fname = name + '.eps'
    plt.savefig(fname, format='eps', dpi=1000, bbox_inches='tight')
    os.system('epstopdf ' + fname)
    os.system('pdfcrop ' + name + '.pdf')
    os.system('cp ' + name + '-crop.pdf ' + dest + '/' + name + '.pdf')
    os.system('rm ' + fname)
    os.system('rm ' + name + '.pdf')
    os.system('rm ' + name + '-crop.pdf')
    print 'saved ' + dest + '/' + name + '.pdf'

In [ ]:
burnin = 3e6 # in data points 
[wtrajectories, corr2s, wwmap] = rotate_traj_ACF(trajectories,burnin)
#all_plots(NLLs,labels,'lreg',[corr2s[0][:-2],corr2s[1][:-2]],wtrajectories[:-2],wwmap,mbs)

In [ ]:
all_plots(NLLs,labels,'lreg',corr2s,wtrajectories,wwmap,mbs)

In [ ]:
# run samplers
n_epochs=1000
batch_size=100
NLL_factor = 5
init = tf.initialize_all_variables()
sess = tf.Session()
sess.run(init)
total_iter=n_epochs*(N/batch_size)
print('Running ' + str(n_epochs) + ' epochs (' +str(total_iter) + ' iterations) per method')
fW_map = np.ndarray.flatten(W_map)

#W_start = np.random.multivariate_normal(fW_map,cov_map)
W_start = fW_map
W_start = np.expand_dims(W_start,1)
k = 3
step_size = .1
labels = []
trajectories2 = []
NLLs2 = []
mbs = []

# lSBPS
k = 3
print('Running lSBPS, k = ' + str(k))
my_lSBPS = lSBPS(D,N,batch_size, [0,0],k)
my_lSBPS.reset_counter()
NLL_lSBPS_0,acc_lSBPS,samples_lSBPS_0 = runOptimizationExplicit(my_lSBPS,0,W_start)
NLL_lSBPS,samples_lSBPS = generate_SBPS_samples(my_lSBPS,W_start,W,N)
print('Number of bounces: ' + str(my_lSBPS.num_bounce))
num_pacc = (my_lSBPS.p_errs/float(my_lSBPS.num_bounce))
print('percent p(acc)>1: ', num_pacc)
reject = (len(samples_lSBPS[:,0]) - my_lSBPS.num_bounce)/float(len(samples_lSBPS[:,0]))
print('percent rejections: ', reject)
print('percent negative slope: ', my_lSBPS.neg_S/float(len(my_lSBPS.all_Gs)))
print 'total travel time: ', my_lSBPS.total_time 
labels.append('SBPS')
trajectories2.append(samples_lSBPS)
NLLs2.append(NLL_lSBPS)
mbs.append(batch_size)

## Run SGLD
stepsizes = [1,.1,.01]
for i in range(len(stepsizes)):
    print('Running SGLD ', stepsizes )
    total_iter=n_epochs*(N/batch_size)
    my_SGLD = SGLD(N,stepsizes[i])
    NLL_SGLD,acc_SGLD,samples_SGLD = runOptimizationExplicit(my_SGLD,0,W_start)
    labels.append('SGLD ' + str(stepsizes[i]))
    trajectories2.append(samples_SGLD)
    NLLs2.append(NLL_SGLD)
    mbs.append(batch_size)

# Run mSGNHT
batch_size=100
total_iter=n_epochs*(N/batch_size)
my_mSGNHT = mSGNHT(D,N,.1)
print 'Running mSGNHT'
NLL_mSGNHT,acc_mSGNHT,samples_mSGNHT = runOptimizationExplicit(my_mSGNHT,0,W_start)
labels.append('mSGNHT')
trajectories2.append(samples_mSGNHT)
NLLs2.append(NLL_mSGNHT)
mbs.append(batch_size)

# lipschitz BPS
print 'Running lipSBPS'
batch_size=1
total_iter=n_epochs*(N/batch_size)
verbose=False
L=np.sqrt(D)*np.max(np.abs(xData)) # this could be tightened, probably by a factor of 2
my_bouncer = lipschitzSBPS(D,N,L,verbose)
NLL_lipSBPS0,acc_lipSBPS,samples_lipSBPS0 = runOptimizationExplicit(my_bouncer,0,W_start)
NLL_lipSBPS0 = 0
acc_lipSBPS = 0
my_bouncer.all_vs = [-v for v in my_bouncer.all_vs]
NLL_lipSBPS,samples_lipSBPS = generate_SBPS_samples(my_bouncer,W_start,W,N)
labels.append('lipSBPS')
trajectories2.append(samples_lipSBPS)
NLLs2.append(NLL_lipSBPS)
mbs.append(batch_size)

# Run ZZ-SS
batch_size=1
total_iter=n_epochs*(N/batch_size)
logisticRegressionM=np.max(xData)
my_zigzag_ss=ZZ_SS(D,N,logisticRegressionM)
my_zigzag_ss.reset_counter()
print('Running ZZ-SS')
NLL_zz_ss0,acc_zz_ss,samples_zz0 = runOptimizationExplicit(my_zigzag_ss,0,W_start)
acc_zz_ss = 0
NLL_zz_ss0 = 0
print('Number of flips: ' + str(my_zigzag_ss.flipcounter))
NLL_zz_ss,samples_zz = generate_SBPS_samples(my_zigzag_ss,W_start,W,N)
labels.append('ZZ-SS')
trajectories2.append(samples_zz)
NLLs2.append(NLL_zz_ss)
mbs.append(1)

wset=W.assign(W_map)
with sess.as_default():
    sess.run(wset)
print 'MAP assigned, x coordinate is  - ', W_start[0]    
NLL_map = sess.run(NLL, feed_dict={xDataPH: xData, yDataPH: yData})
cond_num = np.max(np.linalg.eig(hessian_map)[0])/float(np.min(np.linalg.eig(hessian_map)[0]))
print 'condition number ', cond_num
stats = [total_iter,cond_num,my_lSBPS.num_bounce,my_lSBPS.p_errs/float(my_lSBPS.num_bounce),\
        (len(samples_lSBPS[:,0]) - my_lSBPS.num_bounce)/float(len(samples_lSBPS[:,0])),\
        my_lSBPS.neg_S/float(len(my_lSBPS.all_Gs)),my_lSBPS.total_time]
        # my_plSBPS.num_bounce,my_plSBPS.p_errs/float(my_plSBPS.num_bounce),\
        #(len(samples_plSBPS[:,0]) - my_plSBPS.num_bounce)/float(len(samples_plSBPS[:,0])),\
        #my_plSBPS.neg_S/float(len(my_plSBPS.all_Gs)),my_plSBPS.total_time]
#np.save('samples_' + str(D),[stats,trajectories,NLLs,labels,NLL_map])
#print 'saved samples'

In [ ]:
[wtrajectories2, corr2s2, wwmap2] = rotate_traj_ACF(trajectories2,0)

In [ ]:
asd

In [ ]:
k = 3
batch_size=100
total_iter=n_epochs*(N/batch_size)
print('Running lSBPS, k = ' + str(k))
my_lSBPS = lSBPS(D,N,batch_size, [0,0],k)
my_lSBPS.reset_counter()
NLL_lSBPS_0,acc_lSBPS,samples_lSBPS_0 = runOptimizationExplicit(my_lSBPS,0,W_start)
NLL_lSBPS,samples_lSBPS = generate_SBPS_samples(my_lSBPS,W_start,W,N)
print('Number of bounces: ' + str(my_lSBPS.num_bounce))
num_pacc = (my_lSBPS.p_errs/float(my_lSBPS.num_bounce))
print('percent p(acc)>1: ', num_pacc)
reject = (len(samples_lSBPS[:,0]) - my_lSBPS.num_bounce)/float(len(samples_lSBPS[:,0]))
print('percent rejections: ', reject)
print('percent negative slope: ', my_lSBPS.neg_S/float(len(my_lSBPS.all_Gs)))
print 'total travel time: ', my_lSBPS.total_time 
trajectories2[0] = samples_lSBPS
NLLs2[0] = NLL_lSBPS
burnin = 0
base_change = np.linalg.eig(cov_map)[1]
corr2s3 = []
inds = [np.linalg.eig(cov_map)[0].argmax(),np.linalg.eig(cov_map)[0].argmin()]
wtrajectories3 = []
for i in range(len(trajectories)):
    wtrajectories3.append(np.dot(trajectories2[i],base_change))
wwmap = np.squeeze(np.dot(np.transpose(W_map),base_change))
for j in range(2):
    corrs3 = [] 
    for i in range(1): 
        #ACF
        sig = wtrajectories3[i][burnin/mbs[i]:,inds[j]] - np.mean(wtrajectories3[i][burnin/mbs[i]:,inds[j]]) 
        corr = np.correlate(sig,sig,'full') 
        corr = corr/np.max(corr)
        corr = corr[corr.shape[0]/2:] 
        corrs3.append(corr)
    corr2s3.append(corrs3)

In [ ]:
rng = 5*total_iter/5
inds = [np.linalg.eig(cov_map)[0].argmax(),np.linalg.eig(cov_map)[0].argmin()]
j = 0
for i in range(len(wtrajectories2)): 
    x = np.asarray(range(len(wtrajectories[i][:rng/mbs[i],:])))
    plt.plot(mbs[i]*x,wtrajectories2[i][:rng/mbs[i],inds[j]])

In [ ]:
all_plots(NLLs[:1] + NLLs[2:],labels[:1]+labels[2:],'lreg', [corr2s2[0][:1] + corr2s2[0][2:],corr2s2[1][:1] + corr2s2[1][2:]]\
          ,wtrajectories[:1] + wtrajectories[2:],wwmap,mbs[:1] + mbs[2:])